# Top 1000 IMDB movies

In [1]:
#importing the libraries needed
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep 
from random import randint
import re
import datetime

In [2]:
# Declaring the list of empty variables so the we can append the data overall.
ranking=[]
movies_name=[]
rating=[]
year=[]
time=[]
metascore=[]
votes=[]
gross=[]
genre=[]
director=[]
stars=[]

In [3]:
# Creating an array of values and passing it in the url for the dynamic webpages
pages=np.arange(1,1000,100)

In [4]:
# Finding each value and appending it in the list variable created before.
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        rankings = store.h3.find('span', class_="lister-item-index unbold text-primary").text.strip('.')
        ranking.append(rankings)
        
        name = store.h3.a.text
        movies_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year_of_release =re.sub('\(I*\)\s*|\(*\)*','',year_of_release)
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text.replace('min','')
        time.append(int(runtime))
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(float(rate))
        
        try:
            meta = store.find('span', class_ = "metascore").text
            metascore.append(float(meta))
        except:
            metascore.append(np.nan)

        casting = store.find('p',class_='').text.split('|')
        
        directors =re.sub(r"(\s*Directors?:)|(\n*)",'',casting[0])
        director.append(directors)
        
        star = re.sub(r"(\s*Stars:)|(\n*)",'',casting[1])
        stars.append(star)
        
        value = store.find_all('span', attrs = {'name': "nv"})

        vote = value[0].text
        votes.append(vote)
        
        if len(value)>1:
            grosses = value[1].text 
            if re.search(r'^\$\d+.\d*M',grosses):
                gross.append(grosses)
            else:
                gross.append(np.nan)
        else:
            gross.append(np.nan)

        genres =  store.p.find("span", class_ = 'genre').text.replace('\n', '').strip()
        genre.append(genres)
        
    

In [5]:
# Making a dataframe from the values that is obtained
movie_list = pd.DataFrame({"Ranking":ranking, "Movie Name": movies_name, "Release Year" : year, "Watch Time": time,"Movie Rating": rating, "Metascore": metascore, "Votes" : votes, "Gross": gross, "Genre":genre, "Directors":director, "Stars":stars })
movie_list.head()



,Ranking,Movie Name,Release Year,Watch Time,Movie Rating,Metascore,Votes,Gross,Genre,Directors,Stars
0,1,The Shawshank Redemption,1994,142,9.3,81.0,"2,612,987",$28.34M,Drama,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,2,The Godfather,1972,175,9.2,100.0,"1,808,435",$134.97M,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke..."
2,3,The Dark Knight,2008,152,9.0,84.0,"2,585,038",$534.86M,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,4,The Lord of the Rings: The Return of the King,2003,201,9.0,94.0,"1,793,421",$377.85M,"Action, Adventure, Drama",Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or..."
4,5,Schindler's List,1993,195,9.0,94.0,"1,327,956",$96.90M,"Biography, Drama, History",Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro..."


In [6]:
# Making ranking column as index
movie_list = movie_list.set_index('Ranking')
movie_list.head()

,Movie Name,Release Year,Watch Time,Movie Rating,Metascore,Votes,Gross,Genre,Directors,Stars
Ranking,,,,,,,,,,
1,The Shawshank Redemption,1994,142,9.3,81.0,"2,612,987",$28.34M,Drama,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
2,The Godfather,1972,175,9.2,100.0,"1,808,435",$134.97M,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke..."
3,The Dark Knight,2008,152,9.0,84.0,"2,585,038",$534.86M,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
4,The Lord of the Rings: The Return of the King,2003,201,9.0,94.0,"1,793,421",$377.85M,"Action, Adventure, Drama",Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or..."
5,Schindler's List,1993,195,9.0,94.0,"1,327,956",$96.90M,"Biography, Drama, History",Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro..."


In [7]:
#Saving the data into excel format
movie_list.to_excel("Top 1000 IMDB movies.xlsx")

In [8]:
# The data can also be saved into csv format
movie_list.to_csv("Top 1000 IMDB movies.csv")